# Data Analysis

Now that we've got clean data, let's start with some basic financial analysis.

First, let's load our CSV file into a DataFrame, covert our dates, set the index, and check for duplicated rows or missing values.

In [2]:
import pandas as pd

df = pd.read_csv("TSLA_10_clean.csv")
df["Date"] = pd.to_datetime(df["Date"])
df = df.set_index("Date").sort_index().drop_duplicates()

print("Duplicates:", df.duplicated().sum())
print("Missing:", df.isnull().sum().sum())

ValueError: time data "13/08/2020" doesn't match format "%m/%d/%Y", at position 1. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

## Returns

Returns refer to the gain or loss made on an initial investment, often expressed as a percentage. We can use the generic **percentage change** formula here:

$$ (price_{end} - price_{start}) / price_{start} $$

We can apply this to close prices to calculate the simple daily return:

$$ (close price_{today} - close price_{yesterday}) / close price_{yesterday} $$

In [5]:
jan31_closing = df.loc["2024-01-31", "Close"]
jan30_closing = df.loc["2024-01-30", "Close"]

jan31_return = (jan31_closing - jan30_closing) / jan30_closing
print(f"Return on 31 Jan was {jan31_return:.2%}")

NameError: name 'df' is not defined

This simple daily return expresses a loss in value of 2.24% from one day to the next. Notice we leave our return in decimal form, but when we output it we use `f-strings` and `:.2%` to display it as a percentage.

If we wanted to use the above approach to calculate daily returns for each day in our data set, it would take a long time. Let's see how we can use pandas `pct_change()` to make this sort of work easy, by applying our percentage change formula one column at a time.

Notice how the first row in our data has a missing value **NaN** in the new daily return column. This is because our data doesn't have a close price for the day before it!

What to do with this missing value depends on what further analysis we want to do. If we want to carry out simple descriptive statistics like compute the mean, max, or standard deviation,  we can leave our missing value as NaN, because pandas will by default ignore NaNs when calculating these.

For more complex analyses though, we may want to drop or fill this value. Let's calculate cumulative returns for the period. Instead of comparing a given day with the day before it, cumulative returns compare a given day with the first day of the period, to indicate how our stock has performed since our initial investment.

We generally fill missing daily returns with a 0, which indicates no change with the day before.

Because we're doing cumulative multiplication, we'll add 1 to the closing price to get a growth factor, so we can compound the return over time. After calculating the cumulative product, we subtract 1 to get back to a return.

### Exercise: Bull or Bear?

Calculate the daily change in trading volume over the data frame. Then calculate the average change in trading volume over the period. Take the same approach we used for calculating daily returns, but considering `Volume` instead of `Close` price.

Then determine the trend in TSLA's stock for **Q1 2024**:

- Rising volume and increasing price might indicate a **bullish** trend (where the uptrend is backed by strong demand and could continue).

- Rising volume and decreasing price might indicate a **bearish** trend (where the downtrend is backed by strong selling pressure and could continue).

- Falling volume on price increase or decrease often indicate a trend is losing strength. It might suggest that momentum is waning and a price reversal is coming.

In [ ]:
## YOUR CODE GOES HERE

## Surges

Surges in price or trading volume can be helpful indicators for traders. One way to define a surge is as an increase on the day before by an amount higher than some defined threshold. That threshold is often defined as some number of standard deviations above the mean. Let's look at price surges of five standard deviations above the mean.

## Moving Averages

Moving averages are a different kind of indicator, one that smooths out small variations in trading data to give a better picture of the overall trend.

A Simple Moving Average (SMA) is one which averages out a price over a specific period. The average is "moving" because when a new day is considered in the period, the oldest date is discarded.

Moving averages can be *fast*, when they cover a short period, or *slow* when they consider a longer period. The longer the period, the more those small variations are smoothed out.

## Volatility

Volatility looks at the degree of variance in a stock, and can be helpful for determining risk. Periods of high standard deviation indicate higher volatility and may suggest a riskier investment.
